In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome()

# Mở trang web Tradingview
driver.get("https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/")

# Số lần nhấn nút "Load More" tối đa (hoặc bạn có thể sử dụng điều kiện khác)
max_loads = 5

loads = 0
data_rows = []

while loads < max_loads:
    try:
        load_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".loadButton-SFwfC2e0")))
        load_more_button.click()
        loads += 1
    except:
        break

# Chờ cho dữ liệu được tải đầy đủ
WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".loadingContent-2CAlGV8F")))

# Tạo biến soup1 từ trang web đã tải
soup1 = BeautifulSoup(driver.page_source, 'html.parser')

# Trích xuất dữ liệu và thêm vào danh sách dòng dữ liệu
row_data = soup1.find_all('tr', class_='row-RdUXZpkv')
for row in row_data:
    row_data = row.find_all('td')
    individual_row = [data.text.strip() for data in row_data]
    if len(individual_row) == 11:  # Kiểm tra xem có đủ dữ liệu không
        data_rows.append(individual_row)

# Tạo DataFrame từ danh sách dòng dữ liệu
data_for_all = pd.DataFrame(data_rows, columns=["Stock", "Price", "Change", "Volume", "Market Cap", "PE Ratio", "EPS", "Dividend", "Yield", "Sector", "Recommendation"])

# Đóng trình duyệt sau khi hoàn thành
driver.quit()

In [74]:
data_for_all

,Stock,Price,Change,Volume,Market Cap,PE Ratio,EPS,Dividend,Yield,Sector,Recommendation
0,"AAgilent Technologies, Inc.D",115.91 USD,0.90%,6.89M,33.914B USD,30.40,3.81 USD,−13.21%,0.76%,Health Technology,Buy
1,AAAlcoa CorporationD,28.85 USD,−4.44%,8.716M,5.148B USD,—,−8.15 USD,−259.86%,1.39%,Non-Energy Minerals,Neutral
2,AACAres Acquisition CorporationD,10.71 USD,−0.09%,19.471K,756.172M USD,—,−0.09 USD,−134.01%,0.00%,Finance,—
3,AACIArmada Acquisition Corp. ID,10.75 USD,0.00%,211,99.097M USD,—,−0.03 USD,86.19%,0.00%,Finance,—
4,AACTAres Acquisition Corporation IID,10.25 USD,0.10%,14.04K,512.5M USD,—,—,—,0.00%,Finance,—
...,...,...,...,...,...,...,...,...,...,...,...
495,AYAtlantica Sustainable Infrastructure plcD,22.88 USD,−0.78%,461.448K,2.655B USD,187.54,0.12 USD,—,7.78%,Utilities,Buy
496,"AYIAcuity Brands, Inc.D",159.59 USD,−0.48%,708.466K,4.978B USD,13.73,11.62 USD,11.81%,0.33%,Producer Manufacturing,Buy
497,"AYROAYRO, Inc.D",0.4854 USD,−0.53%,672.097K,18.315M USD,—,−0.64 USD,22.59%,0.00%,Producer Manufacturing,Strong Buy
498,"AYTUAytu BioPharma, Inc.D",1.62 USD,5.88%,33.445K,6.124M USD,—,−60.38 USD,26.73%,0.00%,Health Technology,Strong Buy


In [75]:
# Danh sách các cột chứa ký tự 'USD'
usd_columns = ["Price", "Market Cap", "EPS"]

# Chuyển đổi cột sang kiểu chuỗi và sau đó loại bỏ chữ 'USD'
for column in usd_columns:
    data_for_all[column] = data_for_all[column].astype(str).str.replace(' USD', '')

# In ra DataFrame sau khi đã loại bỏ 'USD'
print(data_for_all)

                                           Stock   Price  Change    Volume  \
0                   AAgilent Technologies, Inc.D  115.91   0.90%     6.89M   
1                           AAAlcoa CorporationD   28.85  −4.44%    8.716M   
2               AACAres Acquisition CorporationD   10.71  −0.09%   19.471K   
3                AACIArmada Acquisition Corp. ID   10.75   0.00%       211   
4           AACTAres Acquisition Corporation IID   10.25   0.10%    14.04K   
..                                           ...     ...     ...       ...   
495  AYAtlantica Sustainable Infrastructure plcD   22.88  −0.78%  461.448K   
496                      AYIAcuity Brands, Inc.D  159.59  −0.48%  708.466K   
497                              AYROAYRO, Inc.D  0.4854  −0.53%  672.097K   
498                    AYTUAytu BioPharma, Inc.D    1.62   5.88%   33.445K   
499                            AYXAlteryx, Inc.D   34.85  −2.13%    2.289M   

    Market Cap PE Ratio     EPS  Dividend  Yield               

In [76]:
data_for_all.fillna(0, inplace=True)

In [77]:
# Hàm để quy đổi đơn vị
def convert_volume(value):
    if 'M' in value:
        return float(value.replace('M', '')) * 1e6
    elif 'B' in value:
        return float(value.replace('B', '')) * 1e9
    elif 'K' in value:
        return float(value.replace('K', '')) * 1e3
    else:
        return float(value)

# Áp dụng hàm cho cột "Volume"
data_for_all['Volume'] = data_for_all['Volume'].apply(convert_volume)

In [78]:
def convert_market_cap(value):
    if 'T' in value:
        return float(value.replace('T', '')) * 1e12  # Chuyển đổi tỷ thành 10^12
    elif 'B' in value:
        return float(value.replace('B', '')) * 1e9   # Chuyển đổi tỉ thành 10^9
    elif 'M' in value:
        return float(value.replace('M', '')) * 1e6   # Chuyển đổi triệu thành 10^6
    elif 'K' in value:
        return float(value.replace('K', '')) * 1e3   # Chuyển đổi ngàn thành 10^3
    else:
        return float(value)

# Áp dụng hàm cho cột "Market Cap"
data_for_all['Market Cap'] = data_for_all['Market Cap'].apply(convert_market_cap)

In [79]:
data_for_all

,Stock,Price,Change,Volume,Market Cap,PE Ratio,EPS,Dividend,Yield,Sector,Recommendation
0,"AAgilent Technologies, Inc.D",115.91,0.90%,6890000.0,3.391400e+10,30.40,3.81,−13.21%,0.76%,Health Technology,Buy
1,AAAlcoa CorporationD,28.85,−4.44%,8716000.0,5.148000e+09,—,−8.15,−259.86%,1.39%,Non-Energy Minerals,Neutral
2,AACAres Acquisition CorporationD,10.71,−0.09%,19471.0,7.561720e+08,—,−0.09,−134.01%,0.00%,Finance,—
3,AACIArmada Acquisition Corp. ID,10.75,0.00%,211.0,9.909700e+07,—,−0.03,86.19%,0.00%,Finance,—
4,AACTAres Acquisition Corporation IID,10.25,0.10%,14040.0,5.125000e+08,—,—,—,0.00%,Finance,—
...,...,...,...,...,...,...,...,...,...,...,...
495,AYAtlantica Sustainable Infrastructure plcD,22.88,−0.78%,461448.0,2.655000e+09,187.54,0.12,—,7.78%,Utilities,Buy
496,"AYIAcuity Brands, Inc.D",159.59,−0.48%,708466.0,4.978000e+09,13.73,11.62,11.81%,0.33%,Producer Manufacturing,Buy
497,"AYROAYRO, Inc.D",0.4854,−0.53%,672097.0,1.831500e+07,—,−0.64,22.59%,0.00%,Producer Manufacturing,Strong Buy
498,"AYTUAytu BioPharma, Inc.D",1.62,5.88%,33445.0,6.124000e+06,—,−60.38,26.73%,0.00%,Health Technology,Strong Buy


In [ ]:
import matplotlib.pyplot as plt
# Lấy dữ liệu từ cột "Price"
prices = data_for_all["Price"]

# Xử lý giá trị để chỉ lấy số và dấu thập phân, loại bỏ ký tự "USD"
prices = prices.str.replace(' USD', '').str.replace('$', '').astype(float)

# Vẽ biểu đồ cho cột "Price"
plt.figure(figsize=(10, 6))  # Kích thước biểu đồ
plt.plot(prices, marker='o', linestyle='-', color='b')  # Vẽ biểu đồ đường
plt.title('Biểu đồ giá cổ phiếu')  # Tiêu đề biểu đồ
plt.xlabel('Số lần load dữ liệu')  # Nhãn x cho trục hoành
plt.ylabel('Giá')  # Nhãn y cho trục tung
plt.grid(True)  # Hiển thị lưới

# Hiển thị biểu đồ
plt.show()


In [ ]:
# Lấy dữ liệu từ 5 cột tiếp theo
columns_to_plot = ["Change", "Volume", "Market Cap", "PE Ratio", "EPS"]

# Vẽ biểu đồ cho từng cột
plt.figure(figsize=(12, 8))  # Kích thước biểu đồ

for column in columns_to_plot:
    plt.plot(data_for_all[column], marker='o', linestyle='-', label=column)

plt.title('Biểu đồ các cột')  # Tiêu đề biểu đồ
plt.xlabel('Số lần load dữ liệu')  # Nhãn x cho trục hoành
plt.grid(True)  # Hiển thị lưới
plt.legend()  # Hiển thị chú thích

# Hiển thị biểu đồ
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dữ liệu mẫu (ảo tưởng)
volume_data = [500000, 750000, 300000, 400000]
pe_ratio_data = [15, 20, 25, 30]
stock_names = ["Stock A", "Stock B", "Stock C", "Stock D"]

# Tạo biểu đồ pie cho dữ liệu "Volume"
plt.figure(figsize=(8, 8))
plt.pie(volume_data, labels=stock_names, autopct='%1.1f%%', startangle=140)
plt.title('Phân phối Volume của các cổ phiếu')
plt.axis('equal')  # Để biểu đồ tròn

plt.show()

# Tạo biểu đồ pie cho dữ liệu "PE Ratio"
plt.figure(figsize=(8, 8))
plt.pie(pe_ratio_data, labels=stock_names, autopct='%1.1f%%', startangle=140)
plt.title('Phân phối PE Ratio của các cổ phiếu')
plt.axis('equal')  # Để biểu đồ tròn

plt.show()

In [81]:
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome()

# Mở trang web Tradingview
driver.get("https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/")

# Số lần nhấn nút "Load More" tối đa (hoặc bạn có thể sử dụng điều kiện khác)
max_loads = 5  # Thay đổi giá trị này nếu cần

loads = 0
data_rows = []

while loads < max_loads:
    try:
        load_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".loadButton-SFwfC2e0")))
        load_more_button.click()
        loads += 1
    except:
        break

# Chờ cho dữ liệu được tải đầy đủ
WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".loadingContent-2CAlGV8F")))

# Tạo biến soup1 từ trang web đã tải
soup1 = BeautifulSoup(driver.page_source, 'html.parser')

# Trích xuất dữ liệu và thêm vào danh sách dòng dữ liệu
row_data = soup1.find_all('tr', class_='row-RdUXZpkv')
for row in row_data:
    row_data = row.find_all('td')
    individual_row = [data.text.strip() for data in row_data]
    if len(individual_row) == 11:  # Kiểm tra xem có đủ dữ liệu không
        data_rows.append(individual_row)

# Tạo DataFrame từ danh sách dòng dữ liệu
data_for_all = pd.DataFrame(data_rows, columns=["Stock", "Price", "Change", "Volume", "Market Cap", "PE Ratio", "EPS", "Dividend", "Yield", "Sector", "Recommendation"])
data_for_all = data_for_all[data_for_all["Price"].str.contains("USD")]

# Xử lý dữ liệu
def convert_to_float(value):
    if value.strip():  # Kiểm tra nếu giá trị không trống
        return float(re.sub(r'[^\d.]', '', value))
    return np.nan  # Trả về NaN nếu giá trị trống

data_for_all["Price"] = data_for_all["Price"].apply(convert_to_float)
data_for_all["Change"] = data_for_all["Change"].apply(convert_to_float)
data_for_all["Volume"] = data_for_all["Volume"].apply(convert_to_float)
data_for_all["PE Ratio"] = data_for_all["PE Ratio"].apply(convert_to_float)
data_for_all["EPS"] = data_for_all["EPS"].apply(convert_to_float)
data_for_all["Dividend"] = data_for_all["Dividend"].apply(convert_to_float)
data_for_all["Yield"] = data_for_all["Yield"].apply(convert_to_float)

# In ra DataFrame
print(data_for_all)

# Đóng trình duyệt sau khi hoàn thành
driver.quit()


ValueError: could not convert string to float: ''